In [8]:
import sys
import os
import getpass
user = getpass.getuser()
DVUTILS_LOCAL_CLONE_PATH = f"/Users/{user}/Documents/GitHub/dvutils"
sys.path.insert(0, DVUTILS_LOCAL_CLONE_PATH)
from utils_io import *

CENSUS_UTILS_PATH = f"/Users/{user}/Documents/GitHub/hess_application/data_processing/6 - affh"
sys.path.insert(0, CENSUS_UTILS_PATH)
import census

import pandas as pd
import geopandas as gpd

## Pull Census Data

In [10]:
census_api = getpass.getpass()

In [34]:
lep_df = census.pull_acs_5_year_est_data(
    census_api_key=census_api,
    acs_year=2021,
    tbl_prof_type="Detailed",
    table_id="C16001",
)

In [35]:
# Rename columns to human-readable
cols = {
    "C16001_001E": "pop_5_years_over",
    "C16001_002E": "pop_english_only",
    "C16001_003E": "spanish_speaking_total",
    "C16001_004E": "spanish_english_vw",
    "C16001_005E": "spanish_english_lvw",
    "C16001_006E": "french_speaking_total",
    "C16001_007E": "french_english_vw",
    "C16001_008E": "french_english_lvw",
    "C16001_009E": "german_speaking_total",
    "C16001_010E": "german_english_vw",
    "C16001_011E": "german_english_lvw",
    "C16001_012E": "russian_speaking_total",
    "C16001_013E": "russian_english_vw",
    "C16001_014E": "russian_english_lvw",
    "C16001_015E": "other_indo_speaking_total",
    "C16001_016E": "other_indo_english_vw",
    "C16001_017E": "other_indo_english_lvw",
    "C16001_018E": "korean_speaking_total",
    "C16001_019E": "korean_english_vw",
    "C16001_020E": "korean_english_lvw",
    "C16001_021E": "chinese_speaking_total",
    "C16001_022E": "chinese_english_vw",
    "C16001_023E": "chinese_english_lvw",
    "C16001_024E": "vietnamese_speaking_total",
    "C16001_025E": "vietnamese_english_vw",
    "C16001_026E": "vietnamese_english_lvw",
    "C16001_027E": "tagalog_speaking_total",
    "C16001_028E": "tagalog_english_vw",
    "C16001_029E": "tagalog_english_lvw",
    "C16001_030E": "other_asian_speaking_total",
    "C16001_031E": "other_asian_english_vw",
    "C16001_032E": "other_asian_english_lvw",
    "C16001_033E": "arabic_speaking_total",
    "C16001_034E": "arabic_english_vw",
    "C16001_035E": "arabic_english_lvw",
    "C16001_036E": "other_unspec_speaking_total",
    "C16001_037E": "other_unspec_english_vw",
    "C16001_038E": "other_unspec_english_lvw",
}
lep_df.rename(columns=cols, inplace=True)

In [39]:
sum_cols = [
    "pop_5_years_over",
    "pop_english_only",
    "spanish_english_lvw",
    "french_english_lvw",
    "german_english_lvw",
    "russian_english_lvw",
    "other_indo_english_lvw",
    "korean_english_lvw",
    "chinese_english_lvw",
    "vietnamese_english_lvw",
    "tagalog_english_lvw",
    "other_asian_english_lvw",
    "arabic_english_lvw",
    "other_unspec_english_lvw",
]
lep_df[sum_cols].agg("sum").reset_index().rename(
    columns={"index": "language_group", 0: "total_population"}
).sort_values("total_population", ascending=False)

,language_group,total_population
0,pop_5_years_over,7322155
1,pop_english_only,4200802
2,spanish_english_lvw,463480
8,chinese_english_lvw,297553
9,vietnamese_english_lvw,103317
6,other_indo_english_lvw,81386
10,tagalog_english_lvw,80000
11,other_asian_english_lvw,61642
7,korean_english_lvw,27806
5,russian_english_lvw,24751


In [40]:
tracts_gdf = census.pull_census_tracts_geodata(year=2020, cartographic=True)

## Merge tract geodata and census attributes

In [44]:
lep_gdf = pd.merge(tracts_gdf, lep_df, on="tract_geoid")

In [46]:
work_dir = "/Users/jcroff/Library/CloudStorage/Box-Box/DataViz Projects/Adhoc-Spatial-Analysis/Air District Language Access Plan Maps"
lep_gdf.to_file(
    os.path.join(work_dir, "Data", "language_spoken_at_home_acs2017_2021.geojson"),
    driver="GeoJSON",
)